In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from google.colab import drive

drive.mount('/content/drive')

# Dataset Generation
np.random.seed(42)
num_samples = 10000
data = {
    "transaction_amount": np.random.uniform(1, 15000, num_samples),
    "transaction_time": np.random.uniform(0, 24, num_samples),
}

def generate_fraud_label(row):
    amount = row["transaction_amount"]
    time = row["transaction_time"]
    if amount > 2000 and (22 <= time <= 24 or 0 <= time <= 6):
        return 1
    elif amount > 10000:
        return np.random.choice([0, 1], p=[0.5, 0.5])
    elif 22 <= time <= 24 or 0 <= time <= 6:
        return np.random.choice([0, 1], p=[0.5, 0.5])
    return 0

data["fraudulent"] = [generate_fraud_label(row) for _, row in pd.DataFrame(data).iterrows()]

# Data Preparation
df = pd.DataFrame(data)
X = df[["transaction_amount", "transaction_time"]].values
y = df["fraudulent"].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model Construction with Functional API
inputs = tf.keras.Input(shape=(2,), name="inputs")
x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = tf.keras.layers.Dense(32, activation='relu', name='dense_2')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name="fraud_model_v1")

# Compile and Train
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Initialize variables
model.predict(np.zeros((1, 2)))

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(X_test, y_test))

# Model Export
model_dir = '/content/drive/My Drive/fraud_model/4'
os.makedirs(model_dir, exist_ok=True)

@tf.function(input_signature=[tf.TensorSpec(shape=[None, 2], dtype=tf.float32)])
def serve(inputs):
    return {'predictions': model(inputs)}

tf.saved_model.save(
    model,
    model_dir,
    signatures={'serving_default': serve},
    options=tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
)

# Verification
print("\n=== Model Inspection ===")
!saved_model_cli show --dir {model_dir} --all
!saved_model_cli run --dir {model_dir} --tag_set serve --signature_def serving_default \
  --input_exprs "inputs=np.zeros((1, 2))"

print("\n=== Normalization Parameters ===")
print(f"Amount: [{scaler.data_min_[0]}, {scaler.data_max_[0]}]")
print(f"Time:   [{scaler.data_min_[1]}, {scaler.data_max_[1]}]")